In [1]:
import pathlib
import json
import copy

import pandas as pd
import geopandas as gpd
import numpy as np

import dtv_backend.chart_templates
import dtv_backend.charts

In [2]:
path = pathlib.Path('../../dtv_backend/tests/user/2022-09-08-result.json')

In [3]:
with open(path) as f:
    results = json.load(f)

#### KPI trip duration

In [4]:
echart = dtv_backend.charts.trip_duration(results)
print(json.dumps(echart))

{"title": {"text": "Trip duration", "subtext": "Example data"}, "tooltip": {"trigger": "axis", "axisPointer": {"type": "cross"}}, "toolbox": {"show": true, "feature": {"saveAsImage": {}}}, "xAxis": {"type": "category", "boundaryGap": true, "data": [0, 1, 2, 3, 4, 5, 6]}, "yAxis": {"type": "value", "axisLabel": {"formatter": "{value}"}, "axisPointer": {"snap": true}}, "visualMap": {"show": true, "dimension": 1, "pieces": [{"lte": 32.98, "color": "green"}, {"gt": 32.98, "color": "red"}]}, "series": [{"name": "Trip duration", "type": "scatter", "data": [13.091111111111111, 5.010555555555555, 5.010555555555555, 5.010555555555555, 5.010277777777778, 5.010555555555555, 5.010555555555555]}]}


#### KPI duration breakdown

In [5]:
echart = dtv_backend.charts.duration_breakdown(results)
print(json.dumps(echart))

{"tooltip": {"trigger": "item", "formatter": "{b} : {c}h ({d}%)"}, "legend": {"orient": "vertical", "left": "left", "data": ["Sailing", "Load request", "Unload request"]}, "series": [{"name": "Duration breakdown", "type": "pie", "radius": "80%", "center": ["50%", "50%"], "data": [{"value": 8.080555555555556, "name": "Load request"}, {"value": 35.073611111111106, "name": "Sailing"}, {"value": 0.0, "name": "Unload request"}], "emphasis": {"itemStyle": {"shadowBlur": 10, "shadowOffsetX": 0, "shadowColor": "rgba(0, 0, 0, 0.5)"}}}]}
